In [1]:
import numpy as np
print("Hello World!")

Hello World!


In [2]:
import numpy as np
from numpy import array
from sympy import symbols, cos, sin, pi, simplify, sqrt, atan2
from sympy.matrices import Matrix

### Create symbols for joint variables
q1, q2, q3, q4, q5, q6, q7 = symbols('q1:8') # theta_i
d1, d2, d3, d4, d5, d6, d7 = symbols('d1:8')
a0, a1, a2, a3, a4, a5, a6 = symbols('a0:7')
alpha0, alpha1, alpha2, alpha3, alpha4, alpha5, alpha6 = symbols('alpha0:7') 

### KUKA KR210 ###
# DH Parameters
s = {alpha0:     0, a0:      0, d1:  0.75, 
     alpha1: -pi/2, a1:   0.35, d2:     0, q2: q2-pi/2, 
     alpha2:     0, a2:   1.25, d3:     0, 
     alpha3: -pi/2, a3: -0.054, d4:  1.50, 
     alpha4:  pi/2, a4:      0, d5:     0, 
     alpha5: -pi/2, a5:      0, d6:     0, 
     alpha6:     0, a6:      0, d7: 0.303, q7: 0}

### Homogeneous Transforms
# base_link to link1
T0_1 = Matrix([[             cos(q1),            -sin(q1),            0,              a0],
               [ sin(q1)*cos(alpha0), cos(q1)*cos(alpha0), -sin(alpha0), -sin(alpha0)*d1],
               [ sin(q1)*sin(alpha0), cos(q1)*sin(alpha0),  cos(alpha0),  cos(alpha0)*d1],
               [                   0,                   0,            0,               1]])
T0_1 = T0_1.subs(s)

T1_2 = Matrix([[             cos(q2),            -sin(q2),            0,              a1],
               [ sin(q2)*cos(alpha1), cos(q2)*cos(alpha1), -sin(alpha1), -sin(alpha1)*d2],
               [ sin(q2)*sin(alpha1), cos(q2)*sin(alpha1),  cos(alpha1),  cos(alpha1)*d2],
               [                   0,                   0,            0,               1]])
T1_2 = T1_2.subs(s)

T2_3 = Matrix([[             cos(q3),            -sin(q3),            0,              a2],
               [ sin(q3)*cos(alpha2), cos(q3)*cos(alpha2), -sin(alpha2), -sin(alpha2)*d3],
               [ sin(q3)*sin(alpha2), cos(q3)*sin(alpha2),  cos(alpha2),  cos(alpha2)*d3],
               [                   0,                   0,            0,               1]])
T2_3 = T2_3.subs(s)

T3_4 = Matrix([[             cos(q4),            -sin(q4),            0,              a3],
               [ sin(q4)*cos(alpha3), cos(q4)*cos(alpha3), -sin(alpha3), -sin(alpha3)*d4],
               [ sin(q4)*sin(alpha3), cos(q4)*sin(alpha3),  cos(alpha3),  cos(alpha3)*d4],
               [                   0,                   0,            0,               1]])
T3_4 = T3_4.subs(s)

T4_5 = Matrix([[             cos(q5),            -sin(q5),            0,              a4],
               [ sin(q5)*cos(alpha4), cos(q5)*cos(alpha4), -sin(alpha4), -sin(alpha4)*d5],
               [ sin(q5)*sin(alpha4), cos(q5)*sin(alpha4),  cos(alpha4),  cos(alpha4)*d5],
               [                   0,                   0,            0,               1]])
T4_5 = T4_5.subs(s)

T5_6 = Matrix([[             cos(q6),            -sin(q6),            0,              a5],
               [ sin(q6)*cos(alpha5), cos(q6)*cos(alpha5), -sin(alpha5), -sin(alpha5)*d6],
               [ sin(q6)*sin(alpha5), cos(q6)*sin(alpha5),  cos(alpha5),  cos(alpha5)*d6],
               [                   0,                   0,            0,               1]])
T5_6 = T5_6.subs(s)

T6_G = Matrix([[             cos(q7),            -sin(q7),            0,              a6],
               [ sin(q7)*cos(alpha6), cos(q7)*cos(alpha6), -sin(alpha6), -sin(alpha6)*d7],
               [ sin(q7)*sin(alpha6), cos(q7)*sin(alpha6),  cos(alpha6),  cos(alpha6)*d7],
               [                   0,                   0,            0,               1]])
T6_G = T6_G.subs(s)

# Composition of Homogeneous Transforms
T0_2 = simplify(T0_1 * T1_2) # base_link to link2
T0_3 = simplify(T0_2 * T2_3) # base_link to link3
T0_4 = simplify(T0_3 * T3_4) # base_link to link4
T0_5 = simplify(T0_4 * T4_5) # base_link to link5
T0_6 = simplify(T0_5 * T5_6) # base_link to link6
T0_G = simplify(T0_6 * T6_G) # base_link to linkG


# Correction Needed to Account of Orientation Difference Between Definition of 
  # Gripper_Link in URDF versus DH Convention
R_z = Matrix([[     cos(np.pi), -sin(np.pi),             0,   0],
              [     sin(np.pi),  cos(np.pi),             0,   0],
              [              0,           0,             1,   0],
              [              0,           0,             0,   1]])
R_y = Matrix([[  cos(-np.pi/2),           0, sin(-np.pi/2),   0],
              [              0,           1,             0,   0],
              [ -sin(-np.pi/2),           0, cos(-np.pi/2),   0],
              [              0,           0,             0,   1]])
R_corr = simplify(R_z * R_y)


# Total Homogeneous Transform Between Base_link and Gripper_link with
  # Orientation Correction Applied
T_total = simplify(T0_G * R_corr)
T_total = T_total.subs(s)

#### Numerically evaluate transforms (compare this with output of tf_echo!)
print("T0_1 = ", T0_1.evalf(subs={q1: 0, q2: 0, q3: 0, q4: 0, q5: 0, q6: 0}))
print("T0_2 = ", T0_2.evalf(subs={q1: 0, q2: 0, q3: 0, q4: 0, q5: 0, q6: 0}))
print("T0_3 = ", T0_3.evalf(subs={q1: 0, q2: 0, q3: 0, q4: 0, q5: 0, q6: 0}))
print("T0_4 = ", T0_4.evalf(subs={q1: 0, q2: 0, q3: 0, q4: 0, q5: 0, q6: 0}))
print("T0_5 = ", T0_5.evalf(subs={q1: 0, q2: 0, q3: 0, q4: 0, q5: 0, q6: 0}))
print("T0_6 = ", T0_6.evalf(subs={q1: 0, q2: 0, q3: 0, q4: 0, q5: 0, q6: 0}))
print("T0_G = ", T0_G.evalf(subs={q1: 0, q2: 0, q3: 0, q4: 0, q5: 0, q6: 0}))


print("T0_1 = ", T0_1)
print("T1_2 = ", T1_2)
print("T2_3 = ", T2_3)
print("T3_4 = ", T3_4)
print("T4_5 = ", T4_5)
print("T5_6 = ", T5_6)
print("T6_G = ", T6_G)
print("T_total = ", T_total)

print("T0_1 = ", T0_1)
print("T0_2 = ", T0_2)
print("T0_3 = ", T0_3)
print("T0_4 = ", T0_4)
print("T0_5 = ", T0_5)
print("T0_6 = ", T0_6)
print("T0_G = ", T0_G)



R3_6 = simplify(T3_4 * T4_5 * T5_6)[0:2, 0:2]
print("T3_6 = ", R3_6)


T0_1 =  Matrix([[1.00000000000000, 0, 0, 0], [0, 1.00000000000000, 0, 0], [0, 0, 1.00000000000000, 0.750000000000000], [0, 0, 0, 1.00000000000000]])
T0_2 =  Matrix([
[  0, 1.0,   0, 0.35],
[  0,   0, 1.0,    0],
[1.0,   0,   0, 0.75],
[  0,   0,   0,  1.0]])
T0_3 =  Matrix([
[  0, 1.0,   0, 0.35],
[  0,   0, 1.0,    0],
[1.0,   0,   0,  2.0],
[  0,   0,   0,  1.0]])
T0_4 =  Matrix([
[  0,    0, 1.0,  1.85],
[  0, -1.0,   0,     0],
[1.0,    0,   0, 1.946],
[  0,    0,   0,   1.0]])
T0_5 =  Matrix([
[  0, 1.0,   0,  1.85],
[  0,   0, 1.0,     0],
[1.0,   0,   0, 1.946],
[  0,   0,   0,   1.0]])
T0_6 =  Matrix([
[  0,    0, 1.0,  1.85],
[  0, -1.0,   0,     0],
[1.0,    0,   0, 1.946],
[  0,    0,   0,   1.0]])
T0_G =  Matrix([
[  0,    0, 1.0, 2.153],
[  0, -1.0,   0,     0],
[1.0,    0,   0, 1.946],
[  0,    0,   0,   1.0]])
T0_1 =  Matrix([[cos(q1), -sin(q1), 0, 0], [sin(q1), cos(q1), 0, 0], [0, 0, 1, 0.750000000000000], [0, 0, 0, 1]])
T1_2 =  Matrix([[sin(q2), cos(q2), 0, 0.350000000

T0_4 =  Matrix([
[sin(q1)*sin(q4) + sin(q2 + q3)*cos(q1)*cos(q4),  sin(q1)*cos(q4) - sin(q4)*sin(q2 + q3)*cos(q1), cos(q1)*cos(q2 + q3), (1.25*sin(q2) - 0.054*sin(q2 + q3) + 1.5*cos(q2 + q3) + 0.35)*cos(q1)],
[sin(q1)*sin(q2 + q3)*cos(q4) - sin(q4)*cos(q1), -sin(q1)*sin(q4)*sin(q2 + q3) - cos(q1)*cos(q4), sin(q1)*cos(q2 + q3), (1.25*sin(q2) - 0.054*sin(q2 + q3) + 1.5*cos(q2 + q3) + 0.35)*sin(q1)],
[                          cos(q4)*cos(q2 + q3),                           -sin(q4)*cos(q2 + q3),        -sin(q2 + q3),          -1.5*sin(q2 + q3) + 1.25*cos(q2) - 0.054*cos(q2 + q3) + 0.75],
[                                             0,                                               0,                    0,                                                                     1]])
T0_5 =  Matrix([
[(sin(q1)*sin(q4) + sin(q2 + q3)*cos(q1)*cos(q4))*cos(q5) + sin(q5)*cos(q1)*cos(q2 + q3), -(sin(q1)*sin(q4) + sin(q2 + q3)*cos(q1)*cos(q4))*sin(q5) + cos(q1)*cos(q5)*cos(q2 + q3), -sin(q1)*cos(q4) 

In [9]:
s = {alpha0:     0, a0:      0, d1:  0.75, 
     alpha1: -pi/2, a1:   0.35, d2:     0, q2: q2-pi/2, 
     alpha2:     0, a2:   1.25, d3:     0, 
     alpha3: -pi/2, a3: -0.054, d4:  1.50, 
     alpha4:  pi/2, a4:      0, d5:     0, 
     alpha5: -pi/2, a5:      0, d6:     0, 
     alpha6:     0, a6:      0, d7: 0.303, q7: 0}


px = 0.468865 
py = 0.229042 
pz = 2.5848
roll = 0
pitch = 0
yaw =  0

#default position
px = 2.153
py = 0
pz = 1.947
roll = 0
pitch = 0
yaw =  0

# only joint1 rot
px = 2.04299979984
py = 2.34452145027e-07
pz = 1.94599919975
roll = -1.43660628282e-07
pitch = 2.80021619226e-07
yaw =  1.14758778448e-07

px = 2.0429949257
py = -3.01679778915e-07
pz = 1.94600428862
roll = 1.61622991435e-07
pitch = -2.33216160161e-06 
yaw =  -1.47665448602e-07

Rrpy = Matrix([[     cos(pitch)*cos(yaw), sin(roll)*sin(pitch)*cos(yaw)-cos(roll)*sin(yaw),  cos(roll)*sin(pitch)+cos(yaw)*sin(roll)*sin(yaw)],
               [     cos(pitch)*sin(yaw), sin(roll)*sin(pitch)*sin(yaw)+cos(roll)*cos(yaw), -sin(roll)*cos(yaw)+cos(roll)*sin(pitch)*sin(yaw)],
               [             -sin(pitch),                             sin(roll)*cos(pitch),                               cos(roll)*cos(pitch)]])

Rrpy = Rrpy * R_corr[0:3,0:3]
print("Rrpy = ", Rrpy)

wx = px - (d6+d7) * Rrpy[0,2]
wy = py - (d6+d7) * Rrpy[1,2]
wz = pz - (d6+d7) * Rrpy[2,2]
            
theta1 = atan2(wy, wx)

L2 = a2               # arm length of joint2
L35 = sqrt(a3**2 +d4**2) # arm length of joint3
theta31 = atan2(-a3, d4) # arm angle of joint3
print("theta31 = ", theta31.evalf(subs=s))
r = sqrt(wx**2 + wy**2) - a1 # distance between joint1 and a point projected xy-plane from the wrist center
S = wz - d1 # distance along z-axis between joint1 and the wrist center
L25 = sqrt(r**2 + S**2)
D = (L25**2 - L2**2 - L35**2) / (2*L2*L35)
theta32 = atan2(sqrt(1-D**2), D)

theta3 = theta32 - theta31 - pi/2


theta22 = atan2(S, r)
D2 = (L2**2 + L25**2 - L35**2) / (2*L2*L25)
#D2 = (L35**2 - L2**2 - L25**2) / (2*L2*L25)
theta21 = atan2(sqrt(1-D2**2), D2)
#theta2 = atan2(wz-d1, r) - atan2(L3*sin(theta3), L2+L3*cos(theta3))
print("theta21, 22 = ", theta21.evalf(subs=s), theta22.evalf(subs=s))
theta2 = pi/2 - (theta21 + theta22)

theta1 = theta1.evalf(subs=s)
theta3 = theta3.evalf(subs=s)
theta2 = theta2.evalf(subs=s)

print("(px, py, pz, roll, pitch, yaw) = ", px, py, pz, roll, pitch, yaw)
print("(wx, wy, wz) = ", wx.evalf(subs=s), wy.evalf(subs=s), wz.evalf(subs=s))
#print("x,y = ", wx.evalf(subs=s), wy.evalf(subs=s))
print("(L2, L35, r, S, D, theta32) = ", L2.evalf(subs=s), L35.evalf(subs=s), r.evalf(subs=s), S.evalf(subs=s), D.evalf(subs=s), theta32.evalf(subs=s))
print("(theta1, theta2, theta3) = ", theta1, theta2, theta3)

print("T_total = ", T_total.evalf(subs={q1: 0, q2: 0, q3: 0, q4: 0, q5: 0, q6: 0}))
#T_calc = T_total.evalf(subs={q1: 0, q2: 0, q3: 0, q4: 0, q5: 0, q6: 0})
#print("T_total = ", T_total)
print("T_total = ", T_total.evalf(subs={q1: theta1, q2: theta2, q3: theta3, q4: 0, q5: 0, q6: 0}))
T_calc = T_total.evalf(subs={q1: theta1, q2: theta2, q3: theta3, q4: 0, q5: 0, q6: 0})
print("calc(px, py, pz) = ", T_calc[0, 3],T_calc[1, 3],T_calc[2, 3] )
print("error = ", px-T_calc[0, 3],py-T_calc[1, 3],pz-T_calc[2, 3])


Rrpy =  Matrix([
[-2.33216162553522e-6, -1.47665071793528e-7,     0.99999999999727],
[-1.61622647055308e-7,   -0.999999999999976, -1.47665448724063e-7],
[   0.999999999997267,  -1.6162299143456e-7,  2.33216160166912e-6]])
theta31 =  0.0359844600820516
theta21, 22 =  0.948526403752716 0.710522374564897
(px, py, pz, roll, pitch, yaw) =  2.0429949257 -3.01679778915e-07 1.94600428862 1.61622991435e-07 -2.33216160161e-06 -1.47665448602e-07
(wx, wy, wz) =  1.73999492570083 -2.56937147951609e-7 1.94600358197503
(L2, L35, r, S, D, theta32) =  1.25000000000000 1.50097168527591 1.38999492570085 1.19600358197503 -0.120696624159311 1.69178793368844
(theta1, theta2, theta3) =  -1.47665458189839e-7 -0.0882524515227163 0.0850071468114962
T_total =  Matrix([
[ 6.12323399573677e-17, 1.22464679914735e-16,                 -1.0, -0.846],
[-7.49879891330929e-33,                  1.0, 1.22464679914735e-16,      0],
[                  1.0,                    0, 6.12323399573677e-17,  2.553],
[               

In [8]:

R0_6 = T0_6[0:3, 0:3]
R0_3 = T0_3[0:3, 0:3] #.evalf(subs={q1: theta1, q2: theta2, q3: theta3})
R0_3_inv = R0_3 ** -1
R3_6 = simplify(T3_4 * T4_5 * T5_6)

print("Rrpy = ", Rrpy)
#print("T_total = ", T_total)
print("T0_6 = ", T0_6)
print("\nR0_6 = ", R0_6)
print("\nR0_3 = ", R0_3)
print("\nR0_3_inv = ", R0_3_inv)
print("\nR3_6 = ", R3_6)

RHS = R0_3_inv * Rrpy
r11, r12, r13 = RHS[0,0], RHS[0,1], RHS[0,2]
r21, r22, r23 = RHS[1,0], RHS[1,1], RHS[1,2]
r31, r32, r33 = RHS[2,0], RHS[2,1], RHS[2,2]

print("\n\nR3_6 = inv(R0_3) * Rrpy")
print(R3_6[0,2], " = ", RHS[0,2].evalf(subs={q1: theta1, q2: theta2, q3: theta3 }))
print(R3_6[1,2], " = ", RHS[1,2].evalf(subs={q1: theta1, q2: theta2, q3: theta3 }))
print(R3_6[2,2], " = ", RHS[2,2].evalf(subs={q1: theta1, q2: theta2, q3: theta3 }))

theta5 = atan2(sqrt(1-r23**2), r23)
theta4 = atan2(r33, -r13)
theta6 = atan2(-r22, r21)

theta5 = theta5.evalf(subs={q1: theta1, q2: theta2, q3: theta3 })
theta4 = theta4.evalf(subs={q1: theta1, q2: theta2, q3: theta3 })
theta6 = theta6.evalf(subs={q1: theta1, q2: theta2, q3: theta3 })

# q4 = tan2(-R[3,3], R[1,3]), q6 = tan2(R[2,2],-R[2,1])

print("(theta4, theta5, theta6) = ", theta4, theta5, theta6)


beta  = atan2(-r31, sqrt(r11 * r11 + r21 * r21))
gamma = atan2(r32, r33)
alpha = atan2(r21, r11)
beta = beta.evalf(subs={q1: theta1, q2: theta2, q3: theta3 })
gamma = gamma.evalf(subs={q1: theta1, q2: theta2, q3: theta3 })
alpha = alpha.evalf(subs={q1: theta1, q2: theta2, q3: theta3 })

print("(alpha, beta, gammma) = ", alpha, beta, gamma)

T_calc = T_total.evalf(subs={q1: theta1, q2: theta2, q3: theta3, q4: theta4, q5: theta5, q6: theta6})
wx = px - (d6+d7) * Rrpy[0,0]
wy = py - (d6+d7) * Rrpy[1,0]
wz = pz - (d6+d7) * Rrpy[2,0]
px_calc = T_calc[0,3] + (d6+d7) * T_calc[0,0]
py_calc = T_calc[1,3] + (d6+d7) * T_calc[1,0]
pz_calc = T_calc[2,3] + (d6+d7) * T_calc[2,0]
px_calc = px_calc.evalf(subs=s)
py_calc = py_calc.evalf(subs=s)
pz_calc = pz_calc.evalf(subs=s)
print("calc(px, py, pz) = ", px_calc, py_calc, pz_calc )
print("error = ", px-px_calc,py-py_calc,pz-pz_calc)


Rrpy =  Matrix([
[-2.33216162553522e-6, -1.47665071793528e-7,     0.99999999999727],
[-1.61622647055308e-7,   -0.999999999999976, -1.47665448724063e-7],
[   0.999999999997267,  -1.6162299143456e-7,  2.33216160166912e-6]])
T0_6 =  Matrix([
[((sin(q1)*sin(q4) + sin(q2 + q3)*cos(q1)*cos(q4))*cos(q5) + sin(q5)*cos(q1)*cos(q2 + q3))*cos(q6) + (sin(q1)*cos(q4) - sin(q4)*sin(q2 + q3)*cos(q1))*sin(q6), -((sin(q1)*sin(q4) + sin(q2 + q3)*cos(q1)*cos(q4))*cos(q5) + sin(q5)*cos(q1)*cos(q2 + q3))*sin(q6) + (sin(q1)*cos(q4) - sin(q4)*sin(q2 + q3)*cos(q1))*cos(q6), -(sin(q1)*sin(q4) + sin(q2 + q3)*cos(q1)*cos(q4))*sin(q5) + cos(q1)*cos(q5)*cos(q2 + q3), (1.25*sin(q2) - 0.054*sin(q2 + q3) + 1.5*cos(q2 + q3) + 0.35)*cos(q1)],
[((sin(q1)*sin(q2 + q3)*cos(q4) - sin(q4)*cos(q1))*cos(q5) + sin(q1)*sin(q5)*cos(q2 + q3))*cos(q6) - (sin(q1)*sin(q4)*sin(q2 + q3) + cos(q1)*cos(q4))*sin(q6), -((sin(q1)*sin(q2 + q3)*cos(q4) - sin(q4)*cos(q1))*cos(q5) + sin(q1)*sin(q5)*cos(q2 + q3))*sin(q6) - (sin(q1)*sin(q4)*sin(

Rotation about the X-axis by 45-degrees
Matrix([[1.00000000000000, 0, 0], [0, 0.707106781186548, -0.707106781186547], [0, 0.707106781186547, 0.707106781186548]])
Rotation about the y-axis by 45-degrees
Matrix([[0.707106781186548, 0, 0.707106781186547], [0, 1.00000000000000, 0], [-0.707106781186547, 0, 0.707106781186548]])
Rotation about the Z-axis by 30-degrees
Matrix([[0.866025403784439, -0.500000000000000, 0], [0.500000000000000, 0.866025403784439, 0], [0, 0, 1.00000000000000]])
